In [1]:
import os
import tensorflow as tf
import tensorflow_io as tfio
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
# from tensorflow.keras.layers.experimental.preprocessing import *
from tensorflow.keras.optimizers import SGD, Adam
from keras.losses import BinaryCrossentropy
from keras.regularizers import l2

In [2]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
DATA_DIR = 'data'
HDF5_FILE = 'data_tf.hdf5'

In [4]:
BATCH_SIZE = 64
IMG_SHAPE = (96, 96, 4)

In [5]:
data_file = os.path.join(DATA_DIR, HDF5_FILE)
ds_x = tfio.IODataset.from_hdf5(data_file, '/x_train')
ds_y = tfio.IODataset.from_hdf5(data_file, '/y_train')

In [6]:
# convert to float and scale to [0, 1)
ds_x = ds_x.map(lambda x: tf.image.convert_image_dtype(x, tf.float32))
# ds_x = ds_images.map(lambda x: tf.image.per_image_standardization(x))

In [7]:
ds_xy = tf.data.Dataset.zip((ds_x, ds_y))
ds_xy = ds_xy.shuffle(BATCH_SIZE*2)

ds_len = ds_xy.reduce(0, lambda x, _: x + 1).numpy()
train_len = int(0.9 * ds_len)

ds_train = ds_xy.take(train_len)
ds_test = ds_xy.skip(train_len)

ds_len, train_len

(220025, 198022)

In [8]:
data_aug = Sequential([
    RandomFlip("horizontal_and_vertical"),
    # RandomZoom(0.2, 0.2)
])

In [9]:
model = Sequential()

model.add(data_aug)

model.add(Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(1e-4))) #, input_shape=(X_train.shape[1:])))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu', kernel_regularizer=l2(2e-4)))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), padding='same', activation='relu', kernel_regularizer=l2(2e-4)))
model.add(BatchNormalization())
model.add(AveragePooling2D (pool_size=(2, 2)))
model.add(Dropout(0.35))

model.add(Conv2D(128, (3, 3), padding='same', activation='relu', kernel_regularizer=l2(3e-4)))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), padding='same', activation='relu', kernel_regularizer=l2(3e-4)))
model.add(BatchNormalization())
model.add(AveragePooling2D (pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

# model.summary()

In [10]:
ds_train = ds_train.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=1e-3),
              metrics=['accuracy'])

history = model.fit(ds_train,
                    epochs=8,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/8
3095/3095 - 156s - loss: 0.5178 - accuracy: 0.8285 - val_loss: 0.4870 - val_accuracy: 0.8332
Epoch 2/8
3095/3095 - 153s - loss: 0.3700 - accuracy: 0.8815 - val_loss: 0.3354 - val_accuracy: 0.8822
Epoch 3/8
3095/3095 - 154s - loss: 0.2953 - accuracy: 0.9060 - val_loss: 0.7206 - val_accuracy: 0.8111
Epoch 4/8
3095/3095 - 153s - loss: 0.2693 - accuracy: 0.9139 - val_loss: 0.3036 - val_accuracy: 0.8963
Epoch 5/8
3095/3095 - 154s - loss: 0.2591 - accuracy: 0.9180 - val_loss: 0.3632 - val_accuracy: 0.8813
Epoch 6/8
3095/3095 - 154s - loss: 0.2501 - accuracy: 0.9211 - val_loss: 0.3944 - val_accuracy: 0.8720
Epoch 7/8
3095/3095 - 154s - loss: 0.2469 - accuracy: 0.9219 - val_loss: 1.0699 - val_accuracy: 0.7860
Epoch 8/8
3095/3095 - 153s - loss: 0.2436 - accuracy: 0.9229 - val_loss: 0.2845 - val_accuracy: 0.9074


In [11]:
ds_train = ds_train.unbatch()
ds_test = ds_test.unbatch()
ds_train = ds_train.batch(128).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(128).prefetch(tf.data.AUTOTUNE)

model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=2e-4),
              metrics=['accuracy'])

history = model.fit(ds_train,
                    epochs=8,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/8
1548/1548 - 164s - loss: 0.1968 - accuracy: 0.9412 - val_loss: 0.1787 - val_accuracy: 0.9464
Epoch 2/8
1548/1548 - 162s - loss: 0.1873 - accuracy: 0.9439 - val_loss: 0.1920 - val_accuracy: 0.9403
Epoch 3/8
1548/1548 - 162s - loss: 0.1796 - accuracy: 0.9460 - val_loss: 0.1936 - val_accuracy: 0.9381
Epoch 4/8
1548/1548 - 161s - loss: 0.1753 - accuracy: 0.9472 - val_loss: 0.1662 - val_accuracy: 0.9483
Epoch 5/8
1548/1548 - 151s - loss: 0.1719 - accuracy: 0.9478 - val_loss: 0.1666 - val_accuracy: 0.9492
Epoch 6/8
1548/1548 - 151s - loss: 0.1684 - accuracy: 0.9487 - val_loss: 0.1829 - val_accuracy: 0.9408
Epoch 7/8
1548/1548 - 161s - loss: 0.1660 - accuracy: 0.9495 - val_loss: 0.2002 - val_accuracy: 0.9366
Epoch 8/8
1548/1548 - 152s - loss: 0.1640 - accuracy: 0.9497 - val_loss: 0.1723 - val_accuracy: 0.9464


In [12]:
ds_train = ds_train.unbatch()
ds_test = ds_test.unbatch()
ds_train = ds_train.batch(256).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(256).prefetch(tf.data.AUTOTUNE)

model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=5e-5),
              metrics=['accuracy'])

history = model.fit(ds_train,
                    epochs=8,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/8
774/774 - 158s - loss: 0.1505 - accuracy: 0.9550 - val_loss: 0.1448 - val_accuracy: 0.9567
Epoch 2/8
774/774 - 155s - loss: 0.1484 - accuracy: 0.9559 - val_loss: 0.1423 - val_accuracy: 0.9575
Epoch 3/8
774/774 - 152s - loss: 0.1469 - accuracy: 0.9564 - val_loss: 0.1471 - val_accuracy: 0.9567
Epoch 4/8
774/774 - 162s - loss: 0.1449 - accuracy: 0.9567 - val_loss: 0.1368 - val_accuracy: 0.9592
Epoch 5/8
774/774 - 152s - loss: 0.1442 - accuracy: 0.9570 - val_loss: 0.1403 - val_accuracy: 0.9575
Epoch 6/8
774/774 - 153s - loss: 0.1429 - accuracy: 0.9574 - val_loss: 0.1407 - val_accuracy: 0.9578
Epoch 7/8
774/774 - 153s - loss: 0.1430 - accuracy: 0.9575 - val_loss: 0.1426 - val_accuracy: 0.9562
Epoch 8/8
774/774 - 164s - loss: 0.1430 - accuracy: 0.9570 - val_loss: 0.1357 - val_accuracy: 0.9604


In [13]:
ds_train = ds_train.unbatch()
ds_test = ds_test.unbatch()
ds_train = ds_train.batch(64).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(64).prefetch(tf.data.AUTOTUNE)

model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=1e-4),
              metrics=['accuracy'])

history = model.fit(ds_train,
                    epochs=2,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/2
3095/3095 - 166s - loss: 0.1617 - accuracy: 0.9492 - val_loss: 0.1506 - val_accuracy: 0.9540
Epoch 2/2
3095/3095 - 159s - loss: 0.1618 - accuracy: 0.9494 - val_loss: 0.1611 - val_accuracy: 0.9513


In [14]:
ds_train = ds_train.unbatch()
ds_test = ds_test.unbatch()
ds_train = ds_train.batch(256).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(256).prefetch(tf.data.AUTOTUNE)

model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=5e-5),
              metrics=['accuracy'])

history = model.fit(ds_train,
                    epochs=5,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/5
774/774 - 161s - loss: 0.1413 - accuracy: 0.9575 - val_loss: 0.1335 - val_accuracy: 0.9595
Epoch 2/5
774/774 - 157s - loss: 0.1395 - accuracy: 0.9583 - val_loss: 0.1338 - val_accuracy: 0.9603
Epoch 3/5
774/774 - 160s - loss: 0.1380 - accuracy: 0.9591 - val_loss: 0.1399 - val_accuracy: 0.9567
Epoch 4/5
774/774 - 162s - loss: 0.1378 - accuracy: 0.9586 - val_loss: 0.1338 - val_accuracy: 0.9603
Epoch 5/5
774/774 - 166s - loss: 0.1367 - accuracy: 0.9593 - val_loss: 0.1344 - val_accuracy: 0.9597


In [15]:
ds_train = ds_train.unbatch()
ds_test = ds_test.unbatch()
ds_train = ds_train.batch(512).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(512).prefetch(tf.data.AUTOTUNE)

model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=1e-4),
              metrics=['accuracy'])

history = model.fit(ds_train,
                    epochs=2,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/2
387/387 - 176s - loss: 0.1345 - accuracy: 0.9598 - val_loss: 0.1387 - val_accuracy: 0.9579
Epoch 2/2
387/387 - 164s - loss: 0.1346 - accuracy: 0.9598 - val_loss: 0.1402 - val_accuracy: 0.9583


In [16]:
model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=5e-5),
              metrics=['accuracy'])

history = model.fit(ds_train,
                    epochs=5,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/5
387/387 - 162s - loss: 0.1314 - accuracy: 0.9604 - val_loss: 0.1344 - val_accuracy: 0.9585
Epoch 2/5
387/387 - 163s - loss: 0.1321 - accuracy: 0.9607 - val_loss: 0.1376 - val_accuracy: 0.9585
Epoch 3/5
387/387 - 161s - loss: 0.1300 - accuracy: 0.9611 - val_loss: 0.1320 - val_accuracy: 0.9611
Epoch 4/5
387/387 - 164s - loss: 0.1294 - accuracy: 0.9618 - val_loss: 0.1292 - val_accuracy: 0.9618
Epoch 5/5
387/387 - 163s - loss: 0.1290 - accuracy: 0.9621 - val_loss: 0.1310 - val_accuracy: 0.9604
